In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
# Configuration
INPUT_SIZE = 3  # thickness, height, angle (what we want to generate)
CONDITION_SIZE = 1  # bending_stiffness (what we condition on)
HIDDEN_DIM = 64
LATENT_DIM = 8
LEARNING_RATE = 0.001
BATCH_SIZE = 32
NUM_EPOCHS = 100
TEST_SIZE = 0.2
RANDOM_STATE = 42
ALPHA = 1.0
device = "cpu"

In [6]:
df = pd.read_csv(r'../MLP/processed_bending_stiffness.csv')
df = df.drop_duplicates()

X = df[['Thickness', 'Height', 'Angle (deg)']].values
y = df['Bending_Stiffness'].values.reshape(-1, 1)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

# Normalize data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)

# Ensure float
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, 
                        shuffle=True, drop_last=False)

In [7]:
class ConditionalVAE(nn.Module):
    """
    Conditional VAE that generates [thickness, height, angle] 
    given a target bending_stiffness value
    """
    def __init__(self, input_dim=3, condition_dim=1, hidden_dim=64, latent_dim=8):
        super(ConditionalVAE, self).__init__()
        
        # Encoder: takes input + condition
        self.encoder = nn.Sequential(
            nn.Linear(input_dim + condition_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        
        # Latent space
        self.mean_layer = nn.Linear(hidden_dim, latent_dim)
        self.logvar_layer = nn.Linear(hidden_dim, latent_dim)
        
        # Decoder: takes latent + condition
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim + condition_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )
     
    def encode(self, x, condition):
        # Concatenate input with condition
        x_cond = torch.cat([x, condition], dim=1)
        h = self.encoder(x_cond)
        mean = self.mean_layer(h)
        logvar = self.logvar_layer(h)
        return mean, logvar
    
    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std
    
    def decode(self, z, condition):
        # Concatenate latent with condition
        z_cond = torch.cat([z, condition], dim=1)
        return self.decoder(z_cond)
    
    def forward(self, x, condition):
        mean, logvar = self.encode(x, condition)
        z = self.reparameterize(mean, logvar)
        x_hat = self.decode(z, condition)
        return x_hat, mean, logvar

def custom_loss_function(x, x_hat, mean, log_var, target_stiffness, forward_model, scaler_X, alpha=1.0):
    # 1. Standard VAE losses
    reconstruction_loss = nn.functional.mse_loss(x_hat, x, reduction='sum')
    kld = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    
    # 2. Denormalize x_hat for the physics model
    # Extract mean and std from your scikit-learn scaler
    # Converting to tensors for torch-compatible math
    x_mean = torch.tensor(scaler_X.mean_, device=x_hat.device).float()
    x_std = torch.tensor(np.sqrt(scaler_X.var_), device=x_hat.device).float()
    
    # x_denorm = (x_hat * std) + mean
    x_hat_denorm = x_hat * x_std + x_mean
    
    # 3. Proxy/Consistency Loss
    # Now forward_model receives raw units (thickness, height, angle)
    predicted_stiffness = forward_model(x_hat_denorm)
    
    # NOTE: Ensure target_stiffness is also in RAW units if forward_model outputs raw units
    proxy_loss = nn.functional.mse_loss(predicted_stiffness, target_stiffness, reduction='sum')
    
    # 4. Combine losses
    total_loss = reconstruction_loss + kld + (alpha * proxy_loss)
    
    loss_dict = {
        'total': total_loss.item(),
        'reconstruction': reconstruction_loss.item(),
        'kld': kld.item(),
        'proxy': proxy_loss.item()
    }
    
    return total_loss, loss_dict

def train_cvae(model, forward_model, optimizer, train_loader, epochs, alpha, device):
    """Train the conditional VAE"""
    model.train()
    for epoch in range(epochs):
        epoch_losses = {
            'total': 0,
            'reconstruction': 0,
            'kld': 0,
            'proxy': 0
        }
        total_samples = 0
        
        for batch_idx, (x, condition) in enumerate(train_loader):
            x = x.to(device)
            condition = condition.to(device)
            current_batch_size = x.size(0)
            
            optimizer.zero_grad()
            
            # Forward pass through VAE
            x_hat, mean, log_var = model(x, condition)
            
            # Calculate custom loss
            loss, loss_dict = custom_loss_function(x, x_hat, mean, log_var, condition, forward_model, alpha)
            
            # Accumulate losses
            for key in epoch_losses:
                epoch_losses[key] += loss_dict[key]
            total_samples += current_batch_size
            
            # Backward pass
            loss.backward()
            optimizer.step()
        
        # Calculate average losses
        for key in epoch_losses:
            epoch_losses[key] /= total_samples
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}")
            print(f"  Total Loss: {epoch_losses['total']:.6f}")
            print(f"  Reconstruction: {epoch_losses['reconstruction']:.6f}")
            print(f"  KLD: {epoch_losses['kld']:.6f}")
            print(f"  Proxy: {epoch_losses['proxy']:.6f}")
    
    return model

def generate_designs(model, target_stiffness, scaler_X, scaler_y, 
                     n_samples=10, device='cpu'):
    """
    Generate designs given a target bending stiffness
    
    Args:
        model: Trained CVAE model
        target_stiffness: Desired bending stiffness value
        scaler_X: StandardScaler fitted on X (thickness, height, angle)
        scaler_y: StandardScaler fitted on y (bending_stiffness)
        n_samples: Number of design variations to generate
        device: 'cpu' or 'cuda'
    
    Returns:
        numpy array of shape (n_samples, 3) with [thickness, height, angle]
    """
    model.eval()
    
    with torch.no_grad():
        # Normalize the target stiffness
        target_normalized = scaler_y.transform([[target_stiffness]])
        target_tensor = torch.FloatTensor(target_normalized).to(device)
        
        # Repeat for n_samples
        condition = target_tensor.repeat(n_samples, 1)
        
        # Sample from prior distribution
        z = torch.randn(n_samples, model.mean_layer.out_features).to(device)
        
        # Generate designs
        generated = model.decode(z, condition)
        
        # Convert back to numpy and denormalize
        generated_np = generated.cpu().numpy()
        designs = scaler_X.inverse_transform(generated_np)
        
    return designs

def verify_generated_designs(designs, forward_model, scaler_X, scaler_y, 
                            target_stiffness, device='cpu'):
    """
    Verify that generated designs actually produce the target stiffness
    
    Args:
        designs: Generated designs (denormalized)
        forward_model: Pre-trained MLP
        scaler_X, scaler_y: Scalers for normalization
        target_stiffness: Target stiffness value (denormalized)
        device: 'cpu' or 'cuda'
    
    Returns:
        predicted_stiffness: Array of predicted stiffness values
    """
    forward_model.eval()
    
    with torch.no_grad():
        # Normalize designs
        designs_normalized = scaler_X.transform(designs)
        designs_tensor = torch.tensor(designs_normalized, dtype=torch.float32).to(device)
        
        # Predict stiffness
        predicted_normalized = forward_model(designs_tensor)
        
        # Denormalize predictions
        predicted_stiffness = scaler_y.inverse_transform(
            predicted_normalized.cpu().numpy()
        )
        
    return predicted_stiffness.flatten()


In [8]:
forward_model = nn.Sequential(
    nn.Linear(3, 64),  # INPUT_SIZE = 3
    nn.ReLU(),
    nn.Dropout(p=0.2),
    
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    
    nn.Linear(16, 1)
).to(device)

# Load the saved weights
forward_model.load_state_dict(
    torch.load(
        r"C:\Users\mason\Work\CMEC_SandwichPanel\Models\MLP\Parameters_To_Stiffness\model_weight.pth",
        map_location=device
    )
)
forward_model.eval()

C:\Users\mason\AppData\Local\Temp\ipykernel_6636\1461096617.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(


Sequential(
  (0): Linear(in_features=3, out_features=64, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=64, out_features=32, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.2, inplace=False)
  (6): Linear(in_features=32, out_features=16, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.2, inplace=False)
  (9): Linear(in_features=16, out_features=1, bias=True)
)

In [9]:
model = ConditionalVAE(
    input_dim=INPUT_SIZE,
    condition_dim=CONDITION_SIZE,
    hidden_dim=HIDDEN_DIM,
    latent_dim=LATENT_DIM
).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train with custom loss
print("\nTraining Conditional VAE with Forward Model Consistency...")
print("="*60)
model = train_cvae(model, forward_model, optimizer, train_loader, NUM_EPOCHS, ALPHA, device)

# Generate and verify designs
print("\n" + "="*60)
print("GENERATING AND VERIFYING DESIGNS")
print("="*60)

target_stiffness = 500.0  # Example target
n_samples = 10

designs = generate_designs(
    model, 
    target_stiffness, 
    scaler_X, 
    scaler_y, 
    n_samples=n_samples, 
    device=device
)

# Verify designs using forward model
predicted_stiffness = verify_generated_designs(
    designs, forward_model, scaler_X, scaler_y, 
    target_stiffness, device
)

print(f"\nTarget Bending Stiffness: {target_stiffness}")
print("\nGenerated Designs and Verification:")
print("-" * 80)
print(f"{'#':<5} {'Thickness':<12} {'Height':<12} {'Angle':<12} {'Predicted':<15} {'Error':<10}")
print("-" * 80)

errors = []
for i, (design, pred) in enumerate(zip(designs, predicted_stiffness), 1):
    error = abs(pred - target_stiffness)
    errors.append(error)
    print(f"{i:<5} {design[0]:<12.4f} {design[1]:<12.4f} {design[2]:<12.4f} "
          f"{pred:<15.2f} {error:<10.2f}")

print("-" * 80)
print(f"Mean Absolute Error: {np.mean(errors):.2f}")
print(f"Std Dev of Error: {np.std(errors):.2f}")

# %%


Training Conditional VAE with Forward Model Consistency...


AttributeError: 'float' object has no attribute 'mean_'